In [1]:
import numpy as np
import os
import argparse
# import cv2
# from imutils import paths
import json
import glob
import xml.etree.ElementTree as Et
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import shutil

# #Selenium components
# import selenium
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.common.exceptions import TimeoutException

In [10]:
def photo_labeler(image_folder):
    if len(os.listdir(image_folder)) == 1:
        start_num = 0
    else:
        num = []
        for x in os.listdir(image_folder):
            if x == '.DS_Store' or x == 'annotations':
                pass
            else:
                num.append(int(x[:-4]))
        start_num = max(num)
    return start_num

def photo_downloader(num_photos):
    print("Downloading of ~{} photos commencing in...".format(len(photo_urls)))
    print('three')
    time.sleep(1)
    print('two')
    time.sleep(1)
    print('one')
    #loop through the URLs
    for url in photo_urls: #you can change this to read in csv file from above 
        try:
            # try to download the image
            r = requests.get(url, timeout=30) #navigate to page, allot 30 seconds to download photo before skipping

            # save the image to disk
            p = os.path.sep.join([image_folder, "{}.jpg".format(str(num_photos).zfill(8))]) #join image name and file path to save at
            f = open(p, "wb")
            f.write(r.content)
            f.close()

            #update the counter and print 
            print("[INFO] downloaded: {}".format(p))
            num_photos += 1

        # handle if any exceptions are thrown during the download process
        except:
            print("[INFO] error downloading {}...skipping".format(p))

    print('downloaded a total of {} photos'.format(num_photos))

In [36]:
!git clone https://github.com/tensorflow/models.git #clone down Tensorflow repository

Cloning into 'models'...
remote: Counting objects: 19848, done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 19848 (delta 28), reused 41 (delta 12), pack-reused 19771
Receiving objects: 100% (19848/19848), 471.42 MiB | 50.59 MiB/s, done.
Resolving deltas: 100% (11645/11645), done.
Checking connectivity... done.
Checking out files: 100% (2966/2966), done.


# Our workflow for this notebook will be as follows:

## Search Google for images with Selenium -> retrieve URLs with Selenium -> save URLs -> download photos from URLs -> label *images* -> split images into *train* and *test* -> convert XML to CSV files -> create TF_Records

In [6]:
search_terms = str(input())

huracan, lamborghini


In [7]:
url = 'https://images.google.com/?gws_rd=ssl'

driver = webdriver.Chrome() #set driver as Chrome
driver.get(url)# open the web page in the browser:
search_bar = driver.find_element_by_xpath('//*[@id="lst-ib"]') # selects search bar
search_bar.send_keys(search_terms) # types in search terms from above
click = driver.find_element_by_xpath('//*[@id="mKlEF"]') #selects the search button
click.click()#"clicks" on the search button

# Search for medium images only
driver.find_element_by_xpath('//*[@id="hdtb-tls"]').click()
time.sleep(0.5)
driver.find_element_by_xpath('//*[@id="hdtbMenus"]/div/div[2]').click()
time.sleep(0.2)
driver.find_element_by_xpath('//*[@id="isz_m"]/a').click()
time.sleep(1)



driver.execute_script("window.scrollTo(0,document.body.scrollHeight);") #page down command to load more photos
time.sleep(0.5)
driver.execute_script("window.scrollTo(0,document.body.scrollHeight);") #page down command to load more photos
time.sleep(0.5)
driver.execute_script("window.scrollTo(0,document.body.scrollHeight);") #page down command to load more photos
time.sleep(0.5)
driver.execute_script("window.scrollTo(0,document.body.scrollHeight);") #page down command to load more photos
time.sleep(0.5)
driver.execute_script("window.scrollTo(0,document.body.scrollHeight);") #page down command to load more photos
time.sleep(0.5)

try:
    driver.find_element_by_xpath('//*[@id="smb"]').click()
except:
    pass

driver.execute_script("window.scrollTo(0,document.body.scrollHeight);") #page down command to load more photos
time.sleep(0.5)
driver.execute_script("window.scrollTo(0,document.body.scrollHeight);") #page down command to load more photos
time.sleep(0.5)
driver.execute_script("window.scrollTo(0,document.body.scrollHeight);") #page down command to load more photos
time.sleep(0.5)
driver.execute_script("window.scrollTo(0,document.body.scrollHeight);") #page down command to load more photos
time.sleep(0.5)
driver.execute_script("window.scrollTo(0,document.body.scrollHeight);") #page down command to load more photos


photo_urls = driver.find_elements_by_class_name("rg_meta") #finds photos on the page

photo_urls = [BeautifulSoup(i.get_attribute('innerHTML'), 'lxml') for i in photo_urls] #returns html for each photo as JSON

photo_urls = [json.loads(_.text) for _ in photo_urls] #parses the JSON

photo_urls = [(d["ou"]) for d in photo_urls] #selects the url text

photo_df = pd.DataFrame(photo_urls, columns=['urls']) #converts urls to text, this can be exported as "test_urls"

driver.close() #closes chrome so it doesn't stay running in background
driver.close() #closes chrome so it doesn't stay running in background

In [1]:
!mkdir ./models/research/object_detection/data_label #make a folder to put our csv file in

In [51]:
photo_df.to_csv('./models/research/object_detection/data_label/aventador_urls.csv') #save photo to csv for backup

### Downloading our photos from the Dataframe

In [6]:
!mkdir ./models/research/object_detection/images #make an images folder

In [13]:
image_folder = './models/research/object_detection/data_label'

In [ ]:
file_name = 'not_lambos.csv'

if file_name in os.listdir(image_folder):
    photo_df.to_csv(path_or_buf=('./image_urls/'+file_name),header=False,mode='a') #append to csv
else:
    photo_df.to_csv(path_or_buf=('./image_urls/'+file_name))

In [53]:
photo_downloader(num_photos=num_photos)

3
2
1
[INFO] downloaded: ./models/research/object_detection/images//00001197.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001198.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001199.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001200.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001201.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001202.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001203.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001204.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001205.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001206.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001207.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001208.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001209.jpg
[INFO] downloaded: 

[INFO] downloaded: ./models/research/object_detection/images//00001307.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001308.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001309.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001310.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001311.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001312.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001313.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001314.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001315.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001316.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001317.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001318.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001319.jpg
[INFO] downloaded: ./mode

[INFO] downloaded: ./models/research/object_detection/images//00001416.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001417.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001418.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001419.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001420.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001421.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001422.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001423.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001424.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001425.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001426.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001427.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001428.jpg
[INFO] downloaded: ./mode

[INFO] downloaded: ./models/research/object_detection/images//00001525.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001526.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001527.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001528.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001529.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001530.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001531.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001532.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001533.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001534.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001535.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001536.jpg
[INFO] downloaded: ./models/research/object_detection/images//00001537.jpg
[INFO] downloaded: ./mode

# Repeat for any other images you want

## Label Images

Using whichever labeling program you prefer, draw a box around each object you want to detect in each photo. There is no limit on objects, just be consitent with your labeling and don't label every little thing

## After Labeling is Finished

In [4]:
!mkdir ./models/research/object_detection/images/test
!mkdir ./models/research/object_detection/images/train

mkdir: cannot create directory './models/research/object_detection/images/test': File exists
mkdir: cannot create directory './models/research/object_detection/images/train': File exists


In [7]:
train_path = "./models/research/object_detection/images/train/"
test_path = "./models/research/object_detection/images/test/"
images_path = "./models/research/object_detection/images/"
labels_path = "./models/research/object_detection/images/annotations/"
listing = os.listdir(images_path)
labels = os.listdir(labels_path)

In [8]:
test_split = 0.3

In [9]:
photo_dict = pd.DataFrame({"photo" : listing})

In [10]:
image_train, image_test = train_test_split(photo_dict,test_size=test_split, random_state=42)

In [11]:
train_name = [_[:-4] for _ in image_train['photo']]
test_name = [_[:-4] for _ in image_test['photo']]

In [12]:
for _ in labels:
    if _[:-4] in train_name:
        shutil.move("{}{}".format(labels_path,_),"{}{}".format(train_path,_))
    elif _[:-4] in test_name :
        shutil.move("{}{}".format(labels_path,_),"{}{}".format(test_path,_))

In [13]:
for _ in image_train['photo'].values:
    if _[-4:] == '.jpg':
        shutil.move("{}{}".format(images_path,_),"{}{}".format(train_path,_))


In [14]:
for _ in image_test['photo'].values:
    try:
        if _[-4:] == '.jpg':
            shutil.move("{}{}".format(images_path,_),"{}{}".format(test_path,_))
    except:
        pass 


In [4]:
#remove xml files if editing
for file in os.listdir(train_path):
    if '.xml' in file:
        os.remove(path="{}{}".format(train_path,file))

In [5]:
#remove xml files if editing
for file in os.listdir(test_path):
    if '.xml' in file:
        os.remove(path="{}{}".format(test_path,file))

# The next steps are in part 2